In [1]:
# continuation of the previous NLP part
# Using averaged word vectors

import numpy as np
norms = np.sum(weights_0_1 * weights_0_1, axis = 1)
norms.resize(norms.shape[0], 1)
# prenormalized all the word embeddings into a matrix called normed_weights
normed_weights = weights_0_1 * norms 

# Converting each reveiw (list of words) into embeddings using the average approach.
def make_sent_vect(words):
    indices = list(map(lambda x: word2index[x], filter(lambda x: x in word2index, words)))
    return np.mean(normed_weights[indices], axis = 0)

reviews2vectors = list()
for review in tokens: # Tokenized reviews
    reviews2vectors.append(make_sent_vect(review))

reviews2vectors = np.array(reviews2vectors)

def most_similar_reviews(review):
    v = make_sent_vect(review)
    scores = Counter()
    for i, val in enumerate(reviews2vectors.dot(v)):
        scores[i] = val
    most_similar - list()
    
    for idx, score in scores.most_common(3):
        most_similar.append(raw_reviews[idx][0 : 40])
    return most_similar

most_similar_reviews(['boring', 'awful'])

NameError: name 'weights_0_1' is not defined

In [6]:
# Working with Identity Matrix

import numpy as np

a = np.array([1, 2, 3])
b = np.array([0.1, 0.2, 0.3])
c = np.array([-1, -0.5, 0])
d = np.array([0, 0, 0])

identity = np.eye(3)
print(identity)

# this multiplication will do nothing
print(a.dot(identity))
print(b.dot(identity))
print(c.dot(identity))
print(d.dot(identity))

this = np.array([2, 4, 6])
movie = np.array([10, 10, 10])
rocks = np.array([1, 1, 1])

print(this + movie + rocks)
print(this.dot(identity) + movie.dot(identity) + rocks.dot(identity))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1. 2. 3.]
[0.1 0.2 0.3]
[-1.  -0.5  0. ]
[0. 0. 0.]
[13 15 17]
[13. 15. 17.]


In [8]:
# toy example with forward propagation
import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp / np.sum(temp, axis = 1, keepdims = True)

word_vects = {}
word_vects['yankees'] = np.array([0., 0., 0.]) # word embedding
word_vects['bears'] = np.array([0., 0., 0.]) # word embedding
word_vects['braves'] = np.array([0., 0., 0.]) # ...
word_vects['red'] = np.array([0., 0., 0.])
word_vects['sox'] = np.array([0., 0., 0.])
word_vects['lose'] = np.array([0., 0., 0.])
word_vects['defeat'] = np.array([0., 0., 0.])
word_vects['beat'] = np.array([0., 0., 0.])
word_vects['tie'] = np.array([0., 0., 0.]) # word embedding

# Sentence embedding to output classification weights
sent2output = np.random.rand(3, len(word_vects))

# Transition weights
identity = np.eye(3)

# Creates a sentence embedding
layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['sox']
layer_2 = layer_1.dot(identity) + word_vects['defeat']

# Predicts over all vocabulary
pred = softmax(layer_2.dot(sent2output))
print(pred)


[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


In [ ]:
# Targets the one-hot vector for 'yankees'
y = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

pred_delta = pred - y
layer_2_delta = pred_delta.dot([sent2output.T])
defeat_delta = layer_2_delta * 1
layer_1_delta = layer_2_delta.dot(identity.T)
sox_delta = layer_1_delta * 1
layer_0_delta = layer_1_delta.dot(identity.T)
alpha = 0.01
word_vects['red'] -= layer_0_delta * alpha
word_vects['sox'] -= sox_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha
identity -= np.outer(layer_0, layer_1_delta) * alpha
identity -= np.outer(layer_1, layer_2_delta) * alpha
sent2output -= np.outer(layer_2, pred_delta) * alpha


In [ ]:
# get the data using the terminal
# wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
# tar -xvf tasks_1-20_v1-1.tar.gz
import sys, random, math
from collections import Counter
import numpy as np

f = open('tasksv11/en/qal_single-supporting-fact_train.txt', 'r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0 : 1000]:
    tokens.append(line.lower().replace('\n', '').split(' ')[1 : ])

print(tokens[0 : 3])

In [ ]:
# Utility functions
vocab = set()

for sent in tokens:
    for word in sent:
        vocab.add(word)
        
vocab = list(vocab)

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
    
    
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis = 0)

In [ ]:
np.random.seed(1)
embed_size = 10

embed = (np.random.rand(len(vocab), embed_size) - 0.5) * 0.1 # Word embeddings
recurrent = np.eye(embed_size) # Embedding -> embedding (initially the identity matrix)
start = np.zeros(embed_size) # Sentence embedding for an empty sentence
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1 # Embedding -> output weights
one_hot = np.eye(len(vocab)) # One-hot lookups (for the loss function)

In [9]:
# Forwar propagation with arbitrary length

def predict(sent):
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)
    
    loss = 0
    
    preds = list() # Forward propagates
    for target_i in range(len(sent)):
        layer = {}
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder)) # Tries to predict the next term
        loss += -np.log(layer['pred'][sent[target_i]])
        
        # Generates the next hidden state
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)
        
    return layers, loss

In [10]:
# Backpropagation with arbitrary length

for iter in range(30000): # Forward
    alpha = 0.001
    sent = words2indices(tokens[iter % len(tokens)][1 : ])
    layers, loss = predict(sent)
    
    for layer_idx in reversed(range(len(layers))): # Backpropagates
        layer = layers[layer_idx]
        target = sent[layer_idx - 1]
        
        if (layer_idx > 0): # If not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())
            
            # If the last layer, don't pull from a later one, because it doesn't exist
            if (layer_idx == len(layers) - 1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose())
                
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose())
            
    # Updating weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx, layer in enumerate(layers[1 : ]):
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
    
    if (iter % 1000 == 0):
        print("Perplexity: " + str(np.exp(loss / len(sent))))

NameError: name 'words2indices' is not defined

In [ ]:
sent_index = 4

l, _ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])

for i, each_layer in enumerate(l[1 : -1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i + 1]
    pred = vocab[each_layer['pred'].argmax()]
    print('Prev Input: ' + input + (' ' * (12 - len(input))) +
          ' True: ' + true + (' ' * (15 - len(true))) + ' Pred: ' + pred)